In [1]:
import numpy as np
import os 
os.environ['TF_ENABLE_ONEDNN_OPTS']='0'
import tensorflow as tf 
import tensorflow_datasets as tfds
keras = tf.keras

In [2]:
ALL_DATA_DIR = '/media/windows/'
training = ALL_DATA_DIR + 'training/'
validation = ALL_DATA_DIR + 'validation/'
scaled = ALL_DATA_DIR + 'scaled/'
scaled_wo_rotate = ALL_DATA_DIR + "scaled_wo_rotate/"

scaled_wo_rotate_training = scaled_wo_rotate + "training/"
scaled_wo_rotate_validation = scaled_wo_rotate + "validation/"

datFile = training + "shape_predictor_68_face_landmarks.dat"

In [3]:
batch_size = 32 
img_height = 224
img_width = 224 

In [4]:
validation_ds = tf.keras.utils.image_dataset_from_directory(scaled_wo_rotate_validation, labels = 'inferred',
            image_size=(img_height,img_width), batch_size=batch_size, seed=42).prefetch(1)

Found 43733 files belonging to 6 classes.


2022-08-03 03:30:59.681326: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-08-03 03:30:59.689918: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-08-03 03:30:59.692561: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-08-03 03:30:59.695757: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the approp

In [5]:
training_ds = tf.keras.utils.image_dataset_from_directory(scaled_wo_rotate_training, labels = 'inferred',
            image_size=(img_height,img_width), batch_size=batch_size, seed=42).prefetch(1)

Found 349639 files belonging to 6 classes.


In [6]:
base_model = keras.applications.MobileNetV3Small(input_shape=(224,224,3), include_top = False, weights='imagenet')

In [7]:
avg = keras.layers.GlobalAveragePooling2D()(base_model.output)
output = keras.layers.Dense(6, activation='softmax')(avg)
model = keras.Model(inputs=base_model.input, outputs=output)

In [8]:
for layer in base_model.layers:
    layer.trainable = False

In [9]:
model.compile(optimizer="Adam",
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [10]:
model.fit(training_ds, validation_data=validation_ds, epochs=3)

Epoch 1/10


2022-08-03 03:31:26.339036: I tensorflow/stream_executor/cuda/cuda_dnn.cc:384] Loaded cuDNN version 8400


10927/10927 [==============================] - 366s 33ms/step - loss: 1.3643 - accuracy: 0.4575 - val_loss: 1.2885 - val_accuracy: 0.4900
Epoch 2/10
10927/10927 [==============================] - 360s 33ms/step - loss: 1.2846 - accuracy: 0.4895 - val_loss: 1.2791 - val_accuracy: 0.4918
Epoch 3/10
10927/10927 [==============================] - 360s 33ms/step - loss: 1.2755 - accuracy: 0.4929 - val_loss: 1.2814 - val_accuracy: 0.4901
Epoch 4/10
10927/10927 [==============================] - 365s 33ms/step - loss: 1.2723 - accuracy: 0.4945 - val_loss: 1.2767 - val_accuracy: 0.4925
Epoch 5/10
10927/10927 [==============================] - 362s 33ms/step - loss: 1.2705 - accuracy: 0.4955 - val_loss: 1.2798 - val_accuracy: 0.4907
Epoch 6/10
10927/10927 [==============================] - 359s 33ms/step - loss: 1.2700 - accuracy: 0.4955 - val_loss: 1.2702 - val_accuracy: 0.4970
Epoch 7/10
10927/10927 [==============================] - 360s 33ms/step - loss: 1.2696 - accuracy: 0.4957 - val_loss

In [21]:
nwbase = tf.keras.applications.MobileNetV3Small(input_shape = (224,224,3), include_top = False, weights=None)
nwavg = keras.layers.GlobalAveragePooling2D()(nwbase.output)
nwout = keras.layers.Dense(6, activation='softmax')(nwavg)
nwmod = keras.Model(inputs=nwbase.input, outputs=nwout)

In [22]:
nwmod.compile(optimizer="Adam", loss="sparse_categorical_crossentropy", metrics=['accuracy'])
nwmod.fit(training_ds, validation_data=validation_ds, epochs=5)

Epoch 1/5
10927/10927 [==============================] - 727s 66ms/step - loss: 0.9688 - accuracy: 0.6143 - val_loss: 1.0429 - val_accuracy: 0.6214
Epoch 2/5
10927/10927 [==============================] - 725s 66ms/step - loss: 0.8098 - accuracy: 0.6817 - val_loss: 0.9318 - val_accuracy: 0.6417
Epoch 3/5
10927/10927 [==============================] - 725s 66ms/step - loss: 0.7402 - accuracy: 0.7110 - val_loss: 0.7730 - val_accuracy: 0.7039
Epoch 4/5
10927/10927 [==============================] - 725s 66ms/step - loss: 0.6892 - accuracy: 0.7328 - val_loss: 0.8233 - val_accuracy: 0.6860
Epoch 5/5
10927/10927 [==============================] - 725s 66ms/step - loss: 0.6482 - accuracy: 0.7500 - val_loss: 0.7548 - val_accuracy: 0.7142


In [23]:
for layer in base_model.layers:
    layer.trainable = True

In [26]:
opt = keras.optimizers.Adam(learning_rate=1e-5)

In [25]:
es = keras.callbacks.EarlyStopping(patience=3, restore_best_weights=True)

In [27]:
model.compile(loss='sparse_categorical_crossentropy', optimizer=opt, metrics=['accuracy'])
history = model.fit(training_ds, validation_data=validation_ds, epochs=20, callbacks=[es])

Epoch 1/20
  475/10927 [>.............................] - ETA: 10:34 - loss: 0.8637 - accuracy: 0.6563

KeyboardInterrupt: 